# Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Dataset Loading

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # to introduce variation in image size and position
    transforms.RandomHorizontalFlip(),     # to account for variability in image orientations
    transforms.ToTensor(),                 
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  
])

transform_test = transforms.Compose([
    transforms.ToTensor(),                 
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


# CNN Class

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)

        # convolutional layer
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)

        # max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # fully connected layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        #1st set of conv -> relu -> pool
        x = self.pool(F.relu(self.conv1(x)))

        #2nd set of conv -> relu -> pool
        x = self.pool(F.relu(self.conv2(x)))

        #flatten before fully connected layers
        x = x.view(-1, 16 * 5 * 5)

        #fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

cnn =CNN().to(device)
cnn

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)

# Training CNN Model

In [ ]:
for epoch in range(50):  

    runningLoss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = cnn(inputs).to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        runningLoss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, runningLoss / 2000))
            runningLoss = 0.0

print('Finished Training')


[1,  2000] loss: 1.498
[1,  4000] loss: 1.468
[1,  6000] loss: 1.433
[1,  8000] loss: 1.410
[1, 10000] loss: 1.380
[1, 12000] loss: 1.381
[2,  2000] loss: 1.300
[2,  4000] loss: 1.304
[2,  6000] loss: 1.272
[2,  8000] loss: 1.248
[2, 10000] loss: 1.256
[2, 12000] loss: 1.252
[3,  2000] loss: 1.189
[3,  4000] loss: 1.208
[3,  6000] loss: 1.176
[3,  8000] loss: 1.173
[3, 10000] loss: 1.168
[3, 12000] loss: 1.152
[4,  2000] loss: 1.079
[4,  4000] loss: 1.108
[4,  6000] loss: 1.109
[4,  8000] loss: 1.112
[4, 10000] loss: 1.120
[4, 12000] loss: 1.094
[5,  2000] loss: 1.026
[5,  4000] loss: 1.049
[5,  6000] loss: 1.021
[5,  8000] loss: 1.064
[5, 10000] loss: 1.043
[5, 12000] loss: 1.049
[6,  2000] loss: 0.940
[6,  4000] loss: 0.996
[6,  6000] loss: 0.985
[6,  8000] loss: 1.005
[6, 10000] loss: 1.014
[6, 12000] loss: 0.994
[7,  2000] loss: 0.933
[7,  4000] loss: 0.940
[7,  6000] loss: 0.921
[7,  8000] loss: 0.947
[7, 10000] loss: 0.959
[7, 12000] loss: 0.988
[8,  2000] loss: 0.870
[8,  4000] 

# Evaluating CNN Model

In [3]:
cnn.eval()

testLoss = 0.0
classCorrect = list(0. for i in range(10))
classTotal = list(0. for i in range(10))

criterion = nn.CrossEntropyLoss()

with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        testLoss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)

        correctTensor = predicted.eq(labels.data.view_as(predicted))
        correct = np.squeeze(correctTensor.cpu().numpy())

        for i in range(4):  
            label = labels.data[i]
            classCorrect[label] += correct[i].item()
            classTotal[label] += 1

testLoss /= len(testloader.dataset)
print('Test Loss: {:.6f}\n'.format(testLoss))

for i in range(10):
    if classTotal[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * classCorrect[i] / classTotal[i],
            np.sum(classCorrect[i]), np.sum(classTotal[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(classCorrect) / np.sum(classTotal),
    np.sum(classCorrect), np.sum(classTotal)))

Test Loss: 1.025660

Test Accuracy of  plane: 76% (760/1000)
Test Accuracy of    car: 90% (900/1000)
Test Accuracy of   bird: 63% (630/1000)
Test Accuracy of    cat: 56% (560/1000)
Test Accuracy of   deer: 71% (710/1000)
Test Accuracy of    dog: 71% (710/1000)
Test Accuracy of   frog: 85% (850/1000)
Test Accuracy of  horse: 74% (740/1000)
Test Accuracy of   ship: 89% (890/1000)
Test Accuracy of  truck: 81% (810/1000)

Test Accuracy (Overall): 75.6% (7560/10000)


# Now Using Pretrained Model vgg16()

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np
from torchvision import models

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

#  Dataset Loading

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # resize for VGG16 input
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainSet = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=32, shuffle=True, num_workers=2)

testSet = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

testLoader = torch.utils.data.DataLoader(testSet, batch_size=32, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:15<00:00, 11107494.62it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Load VGG16 model

In [5]:
vgg16 = models.vgg16(pretrained=True)

# modifying final layer to match the number of classes in CIFAR 10
vgg16.classifier[6] = nn.Linear(vgg16.classifier[6].in_features, 10)
vgg16 = vgg16.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:04<00:00, 125MB/s]


# Training

In [6]:
numEpochs = 10
for epoch in range(numEpochs):
    vgg16.train()
    runningLoss = 0.0
    for i, data in enumerate(trainLoader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = vgg16(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        runningLoss += loss.item()
        if i % 100 == 99:  # prints every 100 mini batches
            print(f'[Epoch {epoch+1}, Mini-batch {i+1}] loss: {runningLoss / 100:.3f}')
            runningLoss = 0.0

print('Finished Training')

[Epoch 1, Mini-batch 100] loss: 1.188
[Epoch 1, Mini-batch 200] loss: 0.630
[Epoch 1, Mini-batch 300] loss: 0.565
[Epoch 1, Mini-batch 400] loss: 0.460
[Epoch 1, Mini-batch 500] loss: 0.424
[Epoch 1, Mini-batch 600] loss: 0.439
[Epoch 1, Mini-batch 700] loss: 0.427
[Epoch 1, Mini-batch 800] loss: 0.382
[Epoch 1, Mini-batch 900] loss: 0.374
[Epoch 1, Mini-batch 1000] loss: 0.357
[Epoch 1, Mini-batch 1100] loss: 0.359
[Epoch 1, Mini-batch 1200] loss: 0.326
[Epoch 1, Mini-batch 1300] loss: 0.319
[Epoch 1, Mini-batch 1400] loss: 0.313
[Epoch 1, Mini-batch 1500] loss: 0.291
[Epoch 2, Mini-batch 100] loss: 0.246
[Epoch 2, Mini-batch 200] loss: 0.254
[Epoch 2, Mini-batch 300] loss: 0.215
[Epoch 2, Mini-batch 400] loss: 0.212
[Epoch 2, Mini-batch 500] loss: 0.234
[Epoch 2, Mini-batch 600] loss: 0.222
[Epoch 2, Mini-batch 700] loss: 0.209
[Epoch 2, Mini-batch 800] loss: 0.205
[Epoch 2, Mini-batch 900] loss: 0.209
[Epoch 2, Mini-batch 1000] loss: 0.203
[Epoch 2, Mini-batch 1100] loss: 0.230
[Epo

# Evaluating

In [9]:
# class names for CIFAR 10 dataset
classNames = [
    'plane', 'car', 'bird', 'cat',
    'deer', 'dog', 'frog', 'horse',
    'ship', 'truck'
]

vgg16.eval()
testLoss = 0.0
classCorrect = list(0. for i in range(10))
classTotal = list(0. for i in range(10))

with torch.no_grad():
    for data in testLoader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        testLoss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            classCorrect[label] += c[i].item()
            classTotal[label] += 1

testLoss = testLoss / len(testloader.dataset)
print(f'Test Loss: {testLoss:.6f}\n')

for i in range(10):
    if classTotal[i] > 0:
        print(f'Test Accuracy of {classNames[i]}: {100 * classCorrect[i] / classTotal[i]:.2f}% ({int(classCorrect[i])}/{int(classTotal[i])})')
    else:
        print(f'Test Accuracy of {classNames[i]}: N/A (no training examples)')

print(f'\nTest Accuracy (Overall): {100. * np.sum(classCorrect) / np.sum(classTotal):.2f}% ({int(np.sum(classCorrect))}/{int(np.sum(classTotal))})')

Test Loss: 0.278592

Test Accuracy of plane: 95.00% (950/1000)
Test Accuracy of car: 97.10% (971/1000)
Test Accuracy of bird: 88.80% (888/1000)
Test Accuracy of cat: 86.50% (865/1000)
Test Accuracy of deer: 92.00% (920/1000)
Test Accuracy of dog: 90.10% (901/1000)
Test Accuracy of frog: 96.70% (967/1000)
Test Accuracy of horse: 96.60% (966/1000)
Test Accuracy of ship: 96.50% (965/1000)
Test Accuracy of truck: 95.10% (951/1000)

Test Accuracy (Overall): 93.44% (9344/10000)
